In [ ]:
import numpy as np
import imutils
import time
import cv2
import os
from imutils.video import FPS
import dlib
from shubhendu_demo_files.CentroidTracker import CentroidTracker

In [ ]:
confThreshold = 0.5
nmsThreshold = 0.3

LABELS = open("coco.names").read().strip().split("\n")

np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

weights = "yolov3.weights"
configFile = "yolov3.cfg"

print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configFile, weights)
ln = net.getLayerNames()
ln = [ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

print("[INFO] Centroid Tracker initialized...")
ct = CentroidTracker()

vs = cv2.VideoCapture("C:/Users/shubh/Downloads/project_video.mp4")
try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
    else cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))
    print("[INFO] {} total frames in video".format(total))
except:
    print("[INFO] couldn't determine frames in video")
    total = -1

In [ ]:
while True:
    (ret, frame) = vs.read()
    if not ret:
        break
    (H, W) = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    rects = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            
            if confidence > confThreshold:
                label = LABELS[classID]
                
                if label != "car":
                    continue
                
                box = detection[0:4] * np.array([W, H, W, H])
                boxes.append(box.astype("int"))
                (centerX, centerY, width, height) = box.astype("int")
                cv2.rectangle(frame, (int(centerX-(width/2)), int(centerY-(height/2))), 
                              (int(centerX+(width/2)), int(centerY+(height/2))), (0, 255, 0), 2)
    objects = ct.update(boxes)
    
    for(objectID, centroid) in objects.items():
        text = "ID {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
    
cv2.destroyAllWindows()
vs.stop()